<a href="https://colab.research.google.com/github/dhu16/alpaca-trading-bot/blob/main/alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Dual moving average
!pip install alpaca_trade_api
import alpaca_trade_api as tradeapi
import threading
import pandas as pd
from datetime import date
from datetime import timedelta
from decimal import Decimal

     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 204kB 18.2MB/s 


In [ ]:
#Auth
api_key = 'PKQW66RE66GPOJCPVNDY'
api_secret = 'o9HTNZ6JTwCTz3lmwrWUq08xK1klUjtVz9Wj7EYP'
base_url = 'https://paper-api.alpaca.markets'

#instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

def get_data(symbol: str):

    #Single day stock price
    today = date.today()
    current_date = today.strftime("%m-%d-%y")
    NY = 'America/New_York'
    start=pd.Timestamp(current_date, tz=NY).isoformat()
    end=pd.Timestamp(current_date, tz=NY).isoformat()

    api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
    bars = api.get_barset([symbol], 'day', start=start, end=end)
    bar_set = bars.df
    bar_set.index.name = "DateTime"
    bar_set.reset_index(inplace=True)
    dataset = pd.DataFrame({'DateTime': bar_set['DateTime'],
                            'Open': bar_set[symbol]['open'],
                            'High': bar_set[symbol]['high'],
                            'Low': bar_set[symbol]['low'],
                            'Close': bar_set[symbol]['close'],
                            'Volume': bar_set[symbol]['volume']})
    return dataset

#obtain account information
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA25EBFUPF1N',
    'buying_power': '397919.21',
    'cash': '97919.21',
    'created_at': '2020-12-10T04:02:05.321554Z',
    'currency': 'USD',
    'daytrade_count': 1,
    'daytrading_buying_power': '397919.21',
    'equity': '99989.81',
    'id': '1a405059-db7e-4563-9723-6ba183ebeb2b',
    'initial_margin': '1035.3',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '2070.6',
    'maintenance_margin': '621.18',
    'multiplier': '4',
    'pattern_day_trader': False,
    'portfolio_value': '99989.81',
    'regt_buying_power': '197909.02',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})


In [ ]:
df = get_data('AAPL')
print(df.head())

                   DateTime    Open    High     Low   Close    Volume
0 2020-12-11 00:00:00-05:00  122.43  122.76  120.55  121.74  62499125


In [ ]:
s = df['Open']
s

0    122.43
Name: Open, dtype: float64

In [ ]:
# Submit a market order to buy 1 share of Apple at market price
api.submit_order(
    symbol='AAPL',

    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc'
)

In [ ]:
#make this something that takes an input 
ticker= input('Enter ticker: ')

#Create DF of desired company
df = get_data(ticker)

#Isolate stock price
d = df['Open']

amt = input('Amount to be traded: ')
int_amt = int(float(amt) / d[0]) 
#stop_price = float(input('Enter stop loss amount: ')) <---- CONVERT PERCENTAGE INPUT TO DECIMAL


# Submit a market order and assign it a Client Order ID.
api.submit_order(
    symbol=ticker,
    #use stock picker input field
    qty=int_amt,
    #take this from amount to be traded on the app 
    #let's say user said $2000/213= 8 stocks you can purschase round(0) Apple stock price it's 213

    side='buy',
    #use signal
    type='market',
    #use market keep it
    time_in_force='day',
    order_class='bracket',
    take_profit=dict(
        limit_price='305.0', #<-------- CONFUSED ABOUT WHAT THIS MEANS AND WHAT OPTIMAL PARAMETERS ARE
    ),
    #generate order id
    client_order_id='order1',
    stop_loss=dict(
        stop_price=d[0]*0.95,
        limit_price='100', # <-------- CONFUSED ON WHAT OPTIMAL PARAMETERS TO USE
    )

    #for selling at a certain percentage loss of money depends on user input
    #stop_loss={'stop_price': d[0] * 0.95}, SHOULD BE * (1-STOP_PRICE)
    #for selling at a certain profit depends on user input will not be used
    #take_profit={'limit_price': symbol_price * 1.05}
)

# Get our order using its Client Order ID.
my_order = api.get_order_by_client_order_id('order1')
print('Got order #{}'.format(my_order.id))


Enter ticker: AAPL
Amount to be traded: 2000
Got order #56788f85-22fb-43d3-bb45-e341a9adf7a9


In [ ]:
#Libraries
import pandas_datareader as web
import numpy as np

In [ ]:
#Rewriting function for 30 and 100 days
def get_data(symbol: str):

    #Single day stock price
    today = date.today() - timedelta(days=1)
    current_date = today.strftime("%m-%d-%y")
    sdate = date.today() - timedelta(days=149)
    startdate = sdate.strftime("%m-%d-%y")
    NY = 'America/New_York'
    start=pd.Timestamp(startdate, tz=NY).isoformat()
    end=pd.Timestamp(current_date, tz=NY).isoformat()

    api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
    bars = api.get_barset([symbol], 'day', start=start, end=end)
    bar_set = bars.df
    bar_set.index.name = "DateTime"
    bar_set.reset_index(inplace=True)
    dataset = pd.DataFrame({'DateTime': bar_set['DateTime'],
                            'Open': bar_set[symbol]['open'],
                            'High': bar_set[symbol]['high'],
                            'Low': bar_set[symbol]['low'],
                            'Close': bar_set[symbol]['close'],
                            'Volume': bar_set[symbol]['volume']})
    return dataset

dataset = get_data('AAPL')
data = dataset['Close']

#Create SMA for 30 days
SMA30 = pd.DataFrame()
SMA30['Close'] = data.rolling(window = 30).mean()

#Create SMA for 100 days
SMA100 = pd.DataFrame()
SMA100['Close'] = data.rolling(window = 100).mean()


#Function for buy and sell
def buy_sell_SMA(data):
    Buy = []
    Sell = []
    flag = -1 #does the price continue to rise or descrease the flag changes when lines cross

    for i in range(0, len(data)):
        if data['SMA30'][i] > data['SMA100'][i]:
            if flag != 1:
                Buy.append(data['Close'])
                Sell.append(np.nan)
                flag = 1
            else:
                Buy.append(np.nan)
                Sell.append(np.nan)
        elif data['SMA30'][i] < data['SMA100'][i]:
            if flag !=0 :
                Buy.append(np.nan)
                Sell.append(data['Close'])
            else: 
                Buy.append(np.nan)
                Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)


    return (Buy, Sell)


#Create a buy and sell column
a = buy_sell_SMA(data)
data['Buy_Signal_Price'] = a[0]
data['Sell_Signal_Price'] = a[1] 

KeyError: ignored